In [72]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

In [7]:
import team_stats

In [10]:
team_stats.squad_stats("eng", 2024, "keepers")

('https://fbref.com/en/comps/9/2024-2025/keepers/2024-2025-Premier-League-Stats',
                team players_used gk_games gk_games_starts gk_minutes  \
 0           Arsenal            1       22              22      1,980   
 1       Aston Villa            2       22              22      1,980   
 2       Bournemouth            3       22              22      1,980   
 3         Brentford            2       22              22      1,980   
 4          Brighton            2       22              22      1,980   
 5           Chelsea            2       22              22      1,980   
 6    Crystal Palace            1       22              22      1,980   
 7           Everton            1       21              21      1,890   
 8            Fulham            1       22              22      1,980   
 9      Ipswich Town            2       22              22      1,980   
 10   Leicester City            3       22              22      1,980   
 11        Liverpool            3       21